**Table of contents**<a id='toc0_'></a>    
- 1. [Using plumber](#toc1_)    
  - 1.1. [Get All pdf files tables](#toc1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Using plumber](#toc0_)

In [1]:

import pdfplumber
from pprint import pprint
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

## 1.1. <a id='toc1_1_'></a>[Get All pdf files tables](#toc0_)

In [2]:
# get all pdf file in a directory that contain keywords in there names
def get_pdfs_with_keyword(directory, keywords:list[str]):
    pdf_files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                if any(keyword in file for keyword in keywords):
                    pdf_files_list.append(os.path.join(root, file))
    return pdf_files_list

In [3]:
def get_potable_water_data(potable_pdf_reprts):
    assert len(potable_pdf_reprts) > 0, "No pdf files Found!"
    print(f"{len(potable_pdf_reprts)} pdf files found")
    all_df = pd.DataFrame()

    for pdf_file in tqdm(potable_pdf_reprts):
        with pdfplumber.open(pdf_file) as pdf:
            tables = [page.extract_table() for page in pdf.pages[:2]]
            for table in tables[:]:
                df = pd.DataFrame(table)
                if (pdf_file[-8:-6] == "20"):
                    df["date"] = "".join([pdf_file[-14:-8] + pdf_file[-6:-4]])
                    # print("Woooow:","".join([pdf_file[-14:-8] + pdf_file[-6:-4]]))
                else:
                    df["date"] = pdf_file[-12:-4]
                    # print(pdf_file[-12:-4])
                all_df = pd.concat([all_df, df], axis=0)
    return all_df

In [4]:
# Replace 'your_directory_path' with the path to the directory you want to search
directory_path = r'../files'
keywords =['RADEES', 'RADESS']
potable_pdf_reprts = get_pdfs_with_keyword(directory_path, keywords)

pprint(potable_pdf_reprts)

['../files\\Rapport potablité Eau RADESS 01-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 02-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 03-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 04-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 05-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 06-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 07-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 08-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 09-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 10-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 11-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 12-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 13-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 14-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 15-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 16-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 17-12-24.pdf',
 '../files\\Rapport potablité Eau RADESS 18-12-2

In [5]:
water_dataframe: pd.DataFrame = get_potable_water_data(potable_pdf_reprts[:])
water_dataframe

26 pdf files found


  0%|          | 0/26 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,date
0,Hydrocarbures polycycliques aromatiques (HAP),None,None,None,None,None,None,None,None,01-12-24
1,Paramètre au laboratoire,Méthode/Version,Résultat,Unité,LQ,Incertitude\n(%),,VMA*,Appréciation,01-12-24
2,Benzo(b) fluorranthène*,NM ISO 28540 (2014),<LQ,µg/l,"0,01",10,None,"0,1",S,01-12-24
3,Benzo(k) fluorranthène*,None,<LQ,µg/l,"0,01",10,None,"0,1",S,01-12-24
4,Benzo(ghi) pérylène*,None,<LQ,µg/l,"0,01",10,None,"0,1",S,01-12-24
...,...,...,...,...,...,...,...,...,...,...
28,Manganèse* (Mn),None,<LQ,mg/l,"0,0005",18,"0,1",S,NaN,26-12-24
29,Zinc* (Zn),None,"0,012",mg/l,"0,005",18,3,S,NaN,26-12-24
30,Fer* (Fe),NM ISO 11885 (2014),<LQ,mg/l,"0,005",18,"0,3",-,NaN,26-12-24
31,Cyanures,Méthode\nPotentiométrique,<LQ,µg/l,10,15,70,S,NaN,26-12-24


In [6]:
# For a pandas DataFrame
def find_result_in_dataframe(df, value="Résultat"):
    result = df.eq(value)
    if result.any().any():
        index, column = np.where(result)
        return index[0], df.columns[column[0]]
    return None, None
# get index and columns for the first value "Result"
i,c = find_result_in_dataframe(water_dataframe, value="Résultat")
print(i,c)
# drop the first 7 lignes from the dataframe
df: pd.DataFrame = water_dataframe.drop(index=range(i))
df.iloc[0, 0] = "Paramètre"
df.iloc[0, c] = "Résul"
df = df.set_axis(df.iloc[0], axis="columns")

date_col = [col for col in df.columns.dropna() if col.endswith(("23","24", "25", "26", "RADEES"))][0]
df.rename(columns={date_col: "date"}, inplace=True)
df = df[[col for col in df.columns.dropna() if col.startswith(("Param", "Résu", "date"))]]
df.insert(0, "date", df.pop('date'))
try:
    df.drop(columns=['Paramètre(s) microbiologiques'], inplace=True)
except:
    pass
df = df.dropna(subset="Paramètre")
# df.sort_values(by='date', inplace=True)

df.query("Paramètre != ''")
df["Résultat"] = df.pop("Résul")
df = df.dropna(subset=["Résultat"]).query("(Résultat != 'Résultat') and (Résultat != 'Résul')")
df

1 2


1,date,Paramètre,Résultat
2,01-12-24,Benzo(b) fluorranthène*,<LQ
3,01-12-24,Benzo(k) fluorranthène*,<LQ
4,01-12-24,Benzo(ghi) pérylène*,<LQ
5,01-12-24,Indénol(1.2.3-cd) pyrène*,<LQ
6,01-12-24,Benzo(a) pyrène*,<LQ
...,...,...,...
28,26-12-24,Manganèse* (Mn),<LQ
29,26-12-24,Zinc* (Zn),"0,012"
30,26-12-24,Fer* (Fe),<LQ
31,26-12-24,Cyanures,<LQ


In [7]:
print(df.isna().sum().sum())
assert df.date.nunique() == len(potable_pdf_reprts), f"some files are skiped:{df.date.nunique()} # {len(potable_pdf_reprts)}"


0


In [8]:
df.to_excel(r'../outputs/data.xlsx', index=False)